In [1]:
# Cell 1 — Setup
import sqlite3
import pandas as pd
import numpy as np

conn = sqlite3.connect("../inventory.db")
print("Connected ✅")

Connected ✅


In [2]:
# Cell 2 — Build freight summary (one of our three sub-queries)
# Why sub-queries? A single massive JOIN on 100K rows is slow.
# Pre-aggregating first makes the final join fast.

freight_sql = """
SELECT
    VendorNumber,
    SUM(Freight) AS FreightCost
FROM vendor_invoice
GROUP BY VendorNumber
"""

freight_df = pd.read_sql(freight_sql, conn)
print(f"Freight summary: {freight_df.shape}")
print(freight_df.head(8))

Freight summary: (10, 2)
   VendorNumber  FreightCost
0          1001    867831.95
1          1002    862024.01
2          1003    555702.50
3          1004    296833.00
4          1005    297974.75
5          1006    289148.81
6          1007    294536.58
7          1008    281359.73


In [3]:
# Cell 3 — Build purchase summary
# Note the WHERE PurchasePrice > 0 — we're filtering data errors in SQL
# Doing it here (at source) is better than filtering in Python later

purchase_sql = """
SELECT
    p.VendorNumber,
    p.VendorName,
    p.Brand,
    p.Description,
    pp.Volume,
    pp.PurchasePrice,
    SUM(p.Quantity)  AS TotalPurchaseQty,
    SUM(p.Dollars)   AS TotalPurchaseDollars
FROM purchases p
JOIN purchase_prices pp ON p.Brand = pp.Brand
WHERE p.PurchasePrice > 0
GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.Description, pp.Volume, pp.PurchasePrice
"""

purchase_df = pd.read_sql(purchase_sql, conn)
print(f"Purchase summary: {purchase_df.shape}")
print(purchase_df.head())

Purchase summary: (15, 8)
   VendorNumber      VendorName  Brand             Description  Volume  \
0          1001   Samsung India   5001  Samsung 55inch QLED TV   750.0   
1          1001   Samsung India   5002      Samsung Galaxy S24   750.0   
2          1001   Samsung India   5014   Samsung Galaxy Tab S9   750.0   
3          1002  LG Electronics   5003    LG Front Load Washer   750.0   
4          1002  LG Electronics   5004       LG OLED 65inch TV   750.0   

   PurchasePrice  TotalPurchaseQty  TotalPurchaseDollars  
0          38000             35465            1347670000  
1          45000             35172            1582740000  
2          35000             35407            1239245000  
3          22000             35521             781462000  
4          55000             35928            1976040000  


In [4]:
# Cell 4 — Build sales summary

sales_sql = """
SELECT
    VendorNumber,
    Brand,
    SUM(SalesQuantity) AS TotalSalesQty,
    SUM(SalesDollars)  AS TotalSalesDollars,
    AVG(SalesPrice)    AS AvgSalesPrice
FROM sales
GROUP BY VendorNumber, Brand
"""

sales_df = pd.read_sql(sales_sql, conn)
print(f"Sales summary: {sales_df.shape}")
print(sales_df.head())

Sales summary: (15, 5)
   VendorNumber  Brand  TotalSalesQty  TotalSalesDollars  AvgSalesPrice
0          1001   5001          43936       2.855022e+09   64980.565527
1          1001   5002          43997       3.520456e+09   80001.790283
2          1001   5014          43081       2.583408e+09   59960.095726
3          1002   5003          42021       1.638503e+09   39008.919091
4          1002   5004          43503       4.134061e+09   95041.332392


In [5]:
# Cell 5 — Join all three summaries into one final table
# merge() in pandas is the equivalent of SQL JOIN
# how='left' means: keep everything from purchase_df even if no match in sales

vendor_summary = (
    purchase_df
    .merge(sales_df,   on=['VendorNumber','Brand'], how='left')
    .merge(freight_df, on='VendorNumber',           how='left')
)

print(f"Final summary shape: {vendor_summary.shape}")
print(f"Columns: {list(vendor_summary.columns)}")
vendor_summary.head()

Final summary shape: (15, 12)
Columns: ['VendorNumber', 'VendorName', 'Brand', 'Description', 'Volume', 'PurchasePrice', 'TotalPurchaseQty', 'TotalPurchaseDollars', 'TotalSalesQty', 'TotalSalesDollars', 'AvgSalesPrice', 'FreightCost']


,VendorNumber,VendorName,Brand,Description,Volume,PurchasePrice,TotalPurchaseQty,TotalPurchaseDollars,TotalSalesQty,TotalSalesDollars,AvgSalesPrice,FreightCost
0,1001,Samsung India,5001,Samsung 55inch QLED TV,750.0,38000,35465,1347670000,43936,2.855022e+09,64980.565527,867831.95
1,1001,Samsung India,5002,Samsung Galaxy S24,750.0,45000,35172,1582740000,43997,3.520456e+09,80001.790283,867831.95
2,1001,Samsung India,5014,Samsung Galaxy Tab S9,750.0,35000,35407,1239245000,43081,2.583408e+09,59960.095726,867831.95
3,1002,LG Electronics,5003,LG Front Load Washer,750.0,22000,35521,781462000,42021,1.638503e+09,39008.919091,862024.01
4,1002,LG Electronics,5004,LG OLED 65inch TV,750.0,55000,35928,1976040000,43503,4.134061e+09,95041.332392,862024.01


In [6]:
# Cell 6 — Data Cleaning
# Always check for issues BEFORE feature engineering

print("=== NULL CHECK ===")
print(vendor_summary.isnull().sum())
print("\n=== DATA TYPES ===")
print(vendor_summary.dtypes)

=== NULL CHECK ===
VendorNumber            0
VendorName              0
Brand                   0
Description             0
Volume                  0
PurchasePrice           0
TotalPurchaseQty        0
TotalPurchaseDollars    0
TotalSalesQty           0
TotalSalesDollars       0
AvgSalesPrice           0
FreightCost             0
dtype: int64

=== DATA TYPES ===
VendorNumber              int64
VendorName               object
Brand                     int64
Description              object
Volume                  float64
PurchasePrice             int64
TotalPurchaseQty          int64
TotalPurchaseDollars      int64
TotalSalesQty             int64
TotalSalesDollars       float64
AvgSalesPrice           float64
FreightCost             float64
dtype: object


In [7]:
# Cell 7 — Apply cleaning fixes

# Fix 1: Strip whitespace from text columns (a real and common data issue)
vendor_summary['VendorName']  = vendor_summary['VendorName'].str.strip()
vendor_summary['Description'] = vendor_summary['Description'].str.strip()

# Fix 2: Cast Volume to numeric (may have come in as text/object)
vendor_summary['Volume'] = pd.to_numeric(vendor_summary['Volume'], errors='coerce')

# Fix 3: Fill null sales values with 0
# Logic: if no sales record exists, sales = 0 (product bought but not sold yet)
fill_cols = ['TotalSalesQty', 'TotalSalesDollars', 'AvgSalesPrice']
vendor_summary[fill_cols] = vendor_summary[fill_cols].fillna(0)

vendor_summary['FreightCost'] = vendor_summary['FreightCost'].fillna(0)

print("After cleaning:")
print(vendor_summary.isnull().sum())

After cleaning:
VendorNumber            0
VendorName              0
Brand                   0
Description             0
Volume                  0
PurchasePrice           0
TotalPurchaseQty        0
TotalPurchaseDollars    0
TotalSalesQty           0
TotalSalesDollars       0
AvgSalesPrice           0
FreightCost             0
dtype: int64


In [8]:
# Cell 8 — Feature Engineering
# We create NEW columns from existing ones to answer business questions
# These are the metrics that will drive our entire analysis

# GROSS PROFIT: money made after paying the vendor
# If this is negative, we're losing money on that product
vendor_summary['GrossProfit'] = vendor_summary['TotalSalesDollars'] - vendor_summary['TotalPurchaseDollars']

# PROFIT MARGIN %: how much of every rupee/dollar in sales is actually profit
# np.where(condition, value_if_true, value_if_false) — handles division by zero
vendor_summary['ProfitMargin'] = np.where(
    vendor_summary['TotalSalesDollars'] > 0,
    (vendor_summary['GrossProfit'] / vendor_summary['TotalSalesDollars']) * 100,
    np.nan   # can't calculate margin if no sales
)

# STOCK TURNOVER: ratio of units sold to units bought
# = 1.0 → sold exactly what you bought (perfect)
# < 1.0 → you bought more than you sold (inventory piling up — bad)
# > 1.0 → you sold more than you bought (using old stock too — watch this)
vendor_summary['StockTurnover'] = np.where(
    vendor_summary['TotalPurchaseQty'] > 0,
    vendor_summary['TotalSalesQty'] / vendor_summary['TotalPurchaseQty'],
    np.nan
)

# UNIT PURCHASE PRICE: average cost per single unit
vendor_summary['UnitPurchasePrice'] = np.where(
    vendor_summary['TotalPurchaseQty'] > 0,
    vendor_summary['TotalPurchaseDollars'] / vendor_summary['TotalPurchaseQty'],
    np.nan
)

# UNSOLD CAPITAL: money tied up in inventory that hasn't sold
# This is a hidden cost — the cash is locked, not earning anything
vendor_summary['UnsoldUnits']   = (vendor_summary['TotalPurchaseQty'] - vendor_summary['TotalSalesQty']).clip(lower=0)
vendor_summary['UnsoldCapital'] = vendor_summary['UnsoldUnits'] * vendor_summary['PurchasePrice']

print("Feature engineering done ✅")
print(f"\nNew columns added: {['GrossProfit','ProfitMargin','StockTurnover','UnitPurchasePrice','UnsoldUnits','UnsoldCapital']}")
vendor_summary[['Description','GrossProfit','ProfitMargin','StockTurnover','UnsoldCapital']].head(10)

Feature engineering done ✅

New columns added: ['GrossProfit', 'ProfitMargin', 'StockTurnover', 'UnitPurchasePrice', 'UnsoldUnits', 'UnsoldCapital']


,Description,GrossProfit,ProfitMargin,StockTurnover,UnsoldCapital
0,Samsung 55inch QLED TV,1.507352e+09,52.796512,1.238855,0
1,Samsung Galaxy S24,1.937716e+09,55.041616,1.250910,0
2,Samsung Galaxy Tab S9,1.344163e+09,52.030606,1.216737,0
3,LG Front Load Washer,8.570405e+08,52.306329,1.182990,0
4,LG OLED 65inch TV,2.158021e+09,52.200993,1.210838,0
5,LG Window AC 1T,7.620680e+08,54.036493,1.224009,0
6,Sony Bravia 4K 50inch,1.257512e+09,53.216407,1.244710,0
7,Sony WH1000XM5 Headphone,4.624332e+08,63.179354,1.279120,0
8,Bosch Dishwasher,7.521326e+08,53.048277,1.161371,0
9,Philips Air Fryer,2.154546e+08,56.568380,1.151333,0


In [9]:
# Cell 9 — Save back to database AND as CSV
# Saving to DB means anyone with the database can query it instantly
# No need to re-run the expensive join query every time

vendor_summary.to_sql('vendor_sales_summary', conn, if_exists='replace', index=False)
conn.commit()

# Also save as CSV — useful for Power BI, Excel, sharing
import os
os.makedirs("../outputs", exist_ok=True)
vendor_summary.to_csv("../outputs/vendor_sales_summary.csv", index=False)

print(f"✅ Saved vendor_sales_summary to database")
print(f"✅ Saved vendor_sales_summary.csv to outputs/")
print(f"\nFinal table: {vendor_summary.shape[0]} rows × {vendor_summary.shape[1]} columns")

✅ Saved vendor_sales_summary to database
✅ Saved vendor_sales_summary.csv to outputs/

Final table: 15 rows × 18 columns
